In [53]:
import web
import time
import logging
import json
import pandas as pd
import numpy as np
import joblib

In [54]:
model_path = "./"
lgb_r_1d = joblib.load(model_path + 'lgb_r_1d.pkl')
lgb_r_2d = joblib.load(model_path + 'lgb_r_2d.pkl')
lgb_r_3d = joblib.load(model_path + 'lgb_r_3d.pkl')
lgb_r_4d = joblib.load(model_path + 'lgb_r_4d.pkl')
lgb_r_5d = joblib.load(model_path + 'lgb_r_5d.pkl')
lgb_r_6d = joblib.load(model_path + 'lgb_r_6d.pkl')
lgb_r_7d = joblib.load(model_path + 'lgb_r_7d.pkl')
source_lgb_b = joblib.load(model_path + 'source_lgb_b.pkl')

In [114]:
def do_role_info(jsondata):
    src_role_info = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    # 根据角色创建时间类型分组
    role_info_predict = None
    for role_data_type in [0, 1, 2, 3, 4, 5, 6, 7]:
        role_info = src_role_info[src_role_info['role_data_type'] == role_data_type]
        if role_info.size == 0:
            continue
        # 特征选择
        select_features = ["user_id", "cp_server_no", "mgame_id", "cp_role_id", "role_id", "create_role_time",
                           "create_role_date","channel_id", "source_id", "game_id", "media_id", "role_data_type",
                           "role_created_login_num", "role_created_active", "role_created_online", "max_role_level",
                           "ip_num", "pay_num","pay_sum","active_0-8", "active_8-12", "active_12-14", "active_14-18",
                           "active_18-24","pay_grade_1", "pay_grade_2", "pay_grade_3", "pay_grade_4",
                           "pay_grade_5", "pay_grade_6","pay_rate", "pay_avg"]
        role_info = role_info[select_features]
        # 根据角色行为分类
        role_info_pay = role_info[role_info['pay_sum'] != 0]
        role_info_nopay = role_info[role_info['pay_sum'] == 0]
        role_info_nopay_online_n = role_info_nopay[(role_info_nopay['role_created_online'] <= 600) &
                        (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10)]
        role_info_nopay_online_y = role_info_nopay[~((role_info_nopay['role_created_online'] <= 600) &
                       (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10))]

        # 标记角户质量不付费不在线0;不付费在线1;付费2
        role_info_nopay_online_n['role_type'] = 0
        role_info_nopay_online_y['role_type'] = 1
        role_info_pay['role_type'] = 2

        # 付费金额预测
        role_info_nopay_online_n['predict_30_pay'] = 0
        role_info_nopay_online_y['predict_30_pay'] = 0
        drop_features = ['user_id', 'cp_server_no', 'mgame_id', 'cp_role_id', 'role_id', 'create_role_time','create_role_date',
                         'role_type',"channel_id", "source_id", "game_id", "media_id", "role_data_type"]
        if role_info_pay.size != 0:
            if role_data_type == 0 or role_data_type == 1:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(role_info_pay.drop(drop_features, axis=1))) * 2.1
            elif role_data_type == 2:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_2d.predict(role_info_pay.drop(drop_features, axis=1))) * 1.64
            elif role_data_type == 3:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_3d.predict(role_info_pay.drop(drop_features, axis=1))) * 1.54
            elif role_data_type == 4:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_4d.predict(role_info_pay.drop(drop_features, axis=1))) * 1.39
            elif role_data_type == 5:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_5d.predict(role_info_pay.drop(drop_features, axis=1))) * 1.32
            elif role_data_type == 6:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_6d.predict(role_info_pay.drop(drop_features, axis=1))) * 1.29
            elif role_data_type == 7:
                role_info_pay['predict_30_pay'] = np.expm1(lgb_r_7d.predict(role_info_pay.drop(drop_features, axis=1))) * 1.255
        # 数据输出
        if role_info_predict is None:
            role_info_predict = pd.concat([role_info_pay, role_info_nopay_online_n, role_info_nopay_online_y], axis=0)
        else:
            role_info_predict = pd.concat([role_info_predict, role_info_pay, role_info_nopay_online_n,
                                           role_info_nopay_online_y], axis=0)
        role_info_predict = role_info_predict[['user_id', 'cp_server_no', 'mgame_id', 'cp_role_id', 'create_role_time','create_role_date',
                         'role_type',"channel_id", "source_id", "game_id", "media_id", "role_data_type",'predict_30_pay']]
    return {"columns": role_info_predict.columns.values.tolist(), "list": role_info_predict.values.tolist()}

In [122]:
jsondata = {"action": "role_info",
      "columns": ["user_id", "cp_server_no", "mgame_id", "cp_role_id", "role_id","create_role_time", "create_role_date", "channel_id", "source_id", "game_id","media_id","role_data_type", 
          "role_created_login_num", "role_created_active", "role_created_online", "max_role_level", "ip_num", "pay_num", "pay_sum", 
          "active_0-8", "active_8-12", "active_12-14", "active_14-18", "active_18-24", 
          "pay_grade_1", "pay_grade_2", "pay_grade_3", "pay_grade_4", "pay_grade_5", "pay_grade_6", 
          "pay_rate", "pay_avg"],
      "data": data[0]}

In [137]:
jsondata = {"action": "role_info",
      "columns": ["user_id", "cp_server_no", "mgame_id", "cp_role_id", "role_id","create_role_time", "create_role_date", "channel_id", "source_id", "game_id","media_id","role_data_type", 
          "role_created_login_num", "role_created_active", "role_created_online", "max_role_level", "ip_num", "pay_num", "pay_sum", 
          "active_0-8", "active_8-12", "active_12-14", "active_14-18", "active_18-24", 
          "pay_grade_1", "pay_grade_2", "pay_grade_3", "pay_grade_4", "pay_grade_5", "pay_grade_6", 
          "pay_rate", "pay_avg"],
      "data": [[120234147,
 'S1164',
 1056,
 '19764109',
 14274099,
 '2020-08-18 10:04:43',
 '2020/8/18',
 20638,
 331512,
 1000993,
 16,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0.0]]}

In [138]:
do_role_info(jsondata)

{'columns': ['user_id',
  'cp_server_no',
  'mgame_id',
  'cp_role_id',
  'create_role_time',
  'create_role_date',
  'role_type',
  'channel_id',
  'source_id',
  'game_id',
  'media_id',
  'role_data_type',
  'predict_30_pay'],
 'list': [[120234147,
   'S1164',
   1056,
   '19764109',
   '2020-08-18 10:04:43',
   '2020/8/18',
   0,
   20638,
   331512,
   1000993,
   16,
   2,
   0.0]]}

In [102]:
src_role_info = pd.DataFrame(jsondata['data'],columns=jsondata['columns'])

In [103]:
src_role_info = pd.DataFrame([data[0][0]],columns=jsondata['columns'])


In [104]:
pd.set_option('display.max_columns',None)

In [105]:
src_role_info[src_role_info['role_data_type'] == 0].shape

(1, 32)

In [106]:
role_info = src_role_info[src_role_info['role_data_type'] == 0]

In [107]:
role_info

,user_id,cp_server_no,mgame_id,cp_role_id,role_id,create_role_time,create_role_date,channel_id,source_id,game_id,media_id,role_data_type,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg
0,120234147,S1164,1056,19764109,14274099,2020-08-18 10:04:43,2020/8/18,20638,331512,1000993,16,0,7,1,0,8,1,0,0.0,0,3,0,0,0,0,0,0,0,0,0,0.0,0.0


In [108]:
role_info_pay = role_info[role_info['pay_sum'] != 0]
role_info_nopay = role_info[role_info['pay_sum'] == 0]
role_info_nopay_online_n = role_info_nopay[(role_info_nopay['role_created_online'] <= 600) &
                (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10)]
role_info_nopay_online_y = role_info_nopay[~((role_info_nopay['role_created_online'] <= 600) &
               (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10))]

# 标记角户质量不付费不在线0;不付费在线1;付费2
role_info_nopay_online_n['role_type'] = 0
role_info_nopay_online_y['role_type'] = 1
role_info_pay['role_type'] = 2

In [109]:
role_info_pay

,user_id,cp_server_no,mgame_id,cp_role_id,role_id,create_role_time,create_role_date,channel_id,source_id,game_id,media_id,role_data_type,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_type


In [96]:
select_features = ["user_id", "cp_server_no", "mgame_id", "cp_role_id", "role_id", "create_role_time",
                   "create_role_date","channel_id", "source_id", "game_id", "media_id", "role_data_type",
                   "role_created_login_num", "role_created_active", "role_created_online", "max_role_level",
                   "ip_num", "pay_num","pay_sum","active_0-8", "active_8-12", "active_12-14", "active_14-18",
                   "active_18-24","pay_grade_1", "pay_grade_2", "pay_grade_3", "pay_grade_4",
                   "pay_grade_5", "pay_grade_6","pay_rate", "pay_avg"]
role_info = role_info[select_features]

In [97]:
drop_features = ['user_id', 'cp_server_no', 'mgame_id', 'cp_role_id', 'role_id', 'create_role_time','create_role_date',
                 'role_type',"channel_id", "source_id", "game_id", "media_id", "role_data_type"]

In [101]:
role_info_pay

,user_id,cp_server_no,mgame_id,cp_role_id,role_id,create_role_time,create_role_date,channel_id,source_id,game_id,media_id,role_data_type,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_type


In [99]:
df1 = role_info_pay.drop(drop_features, axis=1)

In [100]:
df1

,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg


In [98]:
lgb_r_1d.predict(role_info_pay.drop(drop_features, axis=1))

ValueError: Input data must be 2 dimensional and non empty.

In [ ]:

# 根据角色行为分类
role_info_pay = role_info[role_info['pay_sum'] != 0]
role_info_nopay = role_info[role_info['pay_sum'] == 0]
role_info_nopay_online_n = role_info_nopay[(role_info_nopay['role_created_online'] <= 600) &
                (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10)]
role_info_nopay_online_y = role_info_nopay[~((role_info_nopay['role_created_online'] <= 600) &
               (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10))]

# 标记角户质量不付费不在线0;不付费在线1;付费2
role_info_nopay_online_n['role_type'] = 0
role_info_nopay_online_y['role_type'] = 1
role_info_pay['role_type'] = 2

# 付费金额预测
role_info_nopay_online_n['predict_30_pay'] = 0
role_info_nopay_online_y['predict_30_pay'] = 0
drop_features = ['user_id', 'cp_server_no', 'mgame_id', 'cp_role_id', 'role_id', 'create_role_time','create_role_date',
                 'role_type',"channel_id", "source_id", "game_id", "media_id", "role_data_type"]
print(role_info_pay.drop(drop_features, axis=1).shape)
print(role_data_type)
if role_data_type == 0 | role_data_type == 1:
    role_info_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(role_info_pay.drop(drop_features, axis=1))) * 2.1

In [91]:
for role_data_type in [0, 1, 2, 3, 4, 5, 6, 7]:
    role_info = src_role_info[src_role_info['role_data_type'] == role_data_type]
    if role_info.size == 0:
        continue
    print(role_info.shape)
    select_features = ["user_id", "cp_server_no", "mgame_id", "cp_role_id", "role_id", "create_role_time",
                       "create_role_date","channel_id", "source_id", "game_id", "media_id", "role_data_type",
                       "role_created_login_num", "role_created_active", "role_created_online", "max_role_level",
                       "ip_num", "pay_num","pay_sum","active_0-8", "active_8-12", "active_12-14", "active_14-18",
                       "active_18-24","pay_grade_1", "pay_grade_2", "pay_grade_3", "pay_grade_4",
                       "pay_grade_5", "pay_grade_6","pay_rate", "pay_avg"]
    role_info = role_info[select_features]
    # 根据角色行为分类
    role_info_pay = role_info[role_info['pay_sum'] != 0]
    role_info_nopay = role_info[role_info['pay_sum'] == 0]
    role_info_nopay_online_n = role_info_nopay[(role_info_nopay['role_created_online'] <= 600) &
                    (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10)]
    role_info_nopay_online_y = role_info_nopay[~((role_info_nopay['role_created_online'] <= 600) &
                   (role_info_nopay['role_created_login_num'] <= 2) & (role_info_nopay['max_role_level'] <= 10))]

    # 标记角户质量不付费不在线0;不付费在线1;付费2
    role_info_nopay_online_n['role_type'] = 0
    role_info_nopay_online_y['role_type'] = 1
    role_info_pay['role_type'] = 2

    # 付费金额预测
    role_info_nopay_online_n['predict_30_pay'] = 0
    role_info_nopay_online_y['predict_30_pay'] = 0
    drop_features = ['user_id', 'cp_server_no', 'mgame_id', 'cp_role_id', 'role_id', 'create_role_time','create_role_date',
                     'role_type',"channel_id", "source_id", "game_id", "media_id", "role_data_type"]
    print(role_info_pay.drop(drop_features, axis=1).shape)
    print(role_data_type)
    if role_data_type == 0 | role_data_type == 1:
        role_info_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(role_info_pay.drop(drop_features, axis=1))) * 2.1
    #         if role_data_type == 0:
    #             role_info_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(role_info_pay.drop(drop_features, axis=1))) * 2.1

    #         elif role_data_type == 1:
    #             role_info_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(role_info_pay.drop(drop_features, axis=1))) * 2.1


    #         print(role_info_pay['predict_30_pay'])

(1, 32)
(0, 20)
0


In [93]:
role_info_pay

,user_id,cp_server_no,mgame_id,cp_role_id,role_id,create_role_time,create_role_date,channel_id,source_id,game_id,media_id,role_data_type,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_type


In [49]:
(role_data_type==0 or role_data_type ==1)

True

In [48]:
(role_data_type==0 | role_data_type ==1)

False

In [44]:
role_data_type

7

In [42]:
role_info_pay['predict_30_pay']

KeyError: 'predict_30_pay'

In [25]:
for role_data_type in [0, 1, 2, 3, 4, 5, 6, 7]:
    print(src_role_info['role_data_type'] == role_data_type)

0    True
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool
0    False
Name: role_data_type, dtype: bool


In [20]:
for role_data_type in [0, 1, 2, 3, 4, 5, 6, 7]:
        role_info = src_role_info[src_role_info['role_data_type'] == role_data_type]

In [21]:
role_info

,user_id,cp_server_no,mgame_id,cp_role_id,role_id,create_role_time,create_role_date,channel_id,source_id,game_id,media_id,role_data_type,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg
